In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tree-detection/data/maple/mature-sugar-maple--1024x683.jpg
/kaggle/input/tree-detection/data/maple/Sugar-Maple_1_FGT.jpg
/kaggle/input/tree-detection/data/maple/0000559_silver-maple_510.jpg
/kaggle/input/tree-detection/data/maple/Acer-saccharum-Sugar-Maple-Mature-Fall-scaled.jpg
/kaggle/input/tree-detection/data/maple/redmaple_pageimage_JanetandPhil_flickr (1).jpg
/kaggle/input/tree-detection/data/maple/6815004908_b7bbea2119_b.jpg
/kaggle/input/tree-detection/data/maple/acer-autumnfest-web-01.jpg
/kaggle/input/tree-detection/data/maple/unnamed-14.jpg
/kaggle/input/tree-detection/data/maple/gettyimages-602009593-612x612.jpg
/kaggle/input/tree-detection/data/maple/Sugar-Maple-Green-Mountain-600x400.jpg
/kaggle/input/tree-detection/data/maple/acer_palmatum_sango_kaku_dayton_fall_landscape_3967mnc.jpg
/kaggle/input/tree-detection/data/maple/maple_sgr_tr_lg.jpg
/kaggle/input/tree-detection/data/maple/4FFB0CB4-34A8-4061-B83C-B38DEBF02562.jpeg
/kaggle/input/tree-detection/data/m

In [3]:
root_path = '/kaggle/input/tree-detection/'
data_dir = os.path.join(root_path, "data")
root_path, data_dir

('/kaggle/input/tree-detection/', '/kaggle/input/tree-detection/data')

In [2]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
import os
import torch.nn as nn
import torch.optim as optim


In [4]:
transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),      # Flip images horizontally with 50% probability
    transforms.RandomRotation(degrees=15),       # Rotate images randomly within a range of ±15 degrees
    transforms.Resize((224,224)),                # Resizing images to be consistant with each other
    transforms.ToTensor(),                       # Convert to PyTorch tensor
])

In [5]:
class CustomImageDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        # Create a mapping from folder name to label
        self.class_to_idx = {
            "eucalyptus": 0,
            "live_oak": 1,
            "maple": 2,
            "queenpalm": 3,
            "sabel_palm": 4,
            "south_magnolia": 5
            }
        
        # Collect all image paths and their corresponding labels
        for folder_name, label in self.class_to_idx.items():
            folder_path = os.path.join(data_dir, folder_name)
            for img_file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_file)
                if img_path.endswith(('.png', '.jpg', '.jpeg', '.jfif', '.webp')):  # Filter for image files
                    self.image_paths.append(img_path)
                    self.labels.append(label)  # Append the label associated with the folder

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        
        # Apply transformations if any
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [7]:
# Initialize the dataset
dataset = CustomImageDataset(data_dir=data_dir, transform=transformations)

# Load the dataset into DataLoader
data_loader = DataLoader(dataset, batch_size=5, shuffle=True)

In [12]:
num_classes = len(dataset.class_to_idx)
num_classes

6

In [8]:
# len(dataset.class_to_idx)


model = models.resnet18(pretrained=True)

# Modify the final fully connected layer to match the number of classes
model.fc = nn.Linear(model.fc.in_features, num_classes)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 99.1MB/s]


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
from PIL import Image

# Training parameters
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')


Epoch 1/20, Loss: 1.4594
Epoch 2/20, Loss: 1.4959
Epoch 3/20, Loss: 1.3008
Epoch 4/20, Loss: 0.9315
Epoch 5/20, Loss: 0.6534
Epoch 6/20, Loss: 0.9093
Epoch 7/20, Loss: 0.9512
Epoch 8/20, Loss: 0.9172
Epoch 9/20, Loss: 0.7626
Epoch 10/20, Loss: 0.7653
Epoch 11/20, Loss: 0.6880
Epoch 12/20, Loss: 0.7912
Epoch 13/20, Loss: 0.6457
Epoch 14/20, Loss: 0.4743
Epoch 15/20, Loss: 0.4775
Epoch 16/20, Loss: 0.8817
Epoch 17/20, Loss: 0.4378
Epoch 18/20, Loss: 0.3124
Epoch 19/20, Loss: 0.2327
Epoch 20/20, Loss: 0.3046


In [11]:
torch.save(model.state_dict(), "tree_detection_model2.pth")

## **Training Using EfficientNet**

In [13]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
import os
import torch.nn as nn
import torch.optim as optim
from PIL import Image

# Define root paths
root_path = '/kaggle/input/tree-detection/'
data_dir = os.path.join(root_path, "data")

# Data transformations with augmentation and normalization
transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize for pretrained model compatibility
])

# Custom dataset class
class CustomImageDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        # Create a mapping from folder name to label
        self.class_to_idx = {
            "eucalyptus": 0,
            "live_oak": 1,
            "maple": 2,
            "queenpalm": 3,
            "sabel_palm": 4,
            "south_magnolia": 5
        }
        
        # Collect all image paths and their corresponding labels
        for folder_name, label in self.class_to_idx.items():
            folder_path = os.path.join(data_dir, folder_name)
            for img_file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_file)
                if img_path.endswith(('.png', '.jpg', '.jpeg', '.jfif', '.webp')):
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Initialize the dataset and DataLoader
dataset = CustomImageDataset(data_dir=data_dir, transform=transformations)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

num_classes = len(dataset.class_to_idx)

# Load EfficientNet model for better accuracy
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer with learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Training parameters
num_epochs = 20

# Fine-tune all layers
for param in model.parameters():
    param.requires_grad = True

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    
    # Step the learning rate scheduler
    scheduler.step()

# Save the model
torch.save(model.state_dict(), "tree_detection_model_optimized.pth")


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 125MB/s] 


Epoch 1/20, Loss: 1.6370
Epoch 2/20, Loss: 0.6404
Epoch 3/20, Loss: 0.1818
Epoch 4/20, Loss: 0.0591
Epoch 5/20, Loss: 0.0151
Epoch 6/20, Loss: 0.0089
Epoch 7/20, Loss: 0.0026
Epoch 8/20, Loss: 0.0728
Epoch 9/20, Loss: 0.0040
Epoch 10/20, Loss: 0.0070
Epoch 11/20, Loss: 0.0076
Epoch 12/20, Loss: 0.0567
Epoch 13/20, Loss: 0.0020
Epoch 14/20, Loss: 0.0029
Epoch 15/20, Loss: 0.0178
Epoch 16/20, Loss: 0.0116
Epoch 17/20, Loss: 0.0016
Epoch 18/20, Loss: 0.0013
Epoch 19/20, Loss: 0.0011
Epoch 20/20, Loss: 0.0345


## **Training and Validation Using EfficientNet**

In [14]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset, random_split
import os
import torch.nn as nn
import torch.optim as optim
from PIL import Image

# Define root paths
root_path = '/kaggle/input/tree-detection/'
data_dir = os.path.join(root_path, "data")

# Data transformations with augmentation and normalization
transformations = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize for pretrained model compatibility
])

# Custom dataset class
class CustomImageDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        # Create a mapping from folder name to label
        self.class_to_idx = {
            "eucalyptus": 0,
            "live_oak": 1,
            "maple": 2,
            "queenpalm": 3,
            "sabel_palm": 4,
            "south_magnolia": 5
        }
        
        # Collect all image paths and their corresponding labels
        for folder_name, label in self.class_to_idx.items():
            folder_path = os.path.join(data_dir, folder_name)
            for img_file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_file)
                if img_path.endswith(('.png', '.jpg', '.jpeg', '.jfif', '.webp')):
                    self.image_paths.append(img_path)
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Initialize the dataset
dataset = CustomImageDataset(data_dir=data_dir, transform=transformations)

# Split the dataset into training and validation sets (80% training, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

num_classes = len(dataset.class_to_idx)

# Load EfficientNet model for better accuracy
model = models.efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer with learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Training parameters
num_epochs = 20

# Fine-tune all layers
for param in model.parameters():
    param.requires_grad = True

# Training and validation loop
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    
    # Validation phase
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_accuracy = 100 * correct / total
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')
    
    # Step the learning rate scheduler
    scheduler.step()

# Save the model
torch.save(model.state_dict(), "tree_detection_model_optimized_eval.pth")


Epoch 1/20, Loss: 1.7422, Validation Accuracy: 66.67%
Epoch 2/20, Loss: 0.8606, Validation Accuracy: 83.33%
Epoch 3/20, Loss: 0.3314, Validation Accuracy: 83.33%
Epoch 4/20, Loss: 0.1019, Validation Accuracy: 94.44%
Epoch 5/20, Loss: 0.0530, Validation Accuracy: 100.00%
Epoch 6/20, Loss: 0.0293, Validation Accuracy: 72.22%
Epoch 7/20, Loss: 0.0807, Validation Accuracy: 83.33%
Epoch 8/20, Loss: 0.0431, Validation Accuracy: 77.78%
Epoch 9/20, Loss: 0.0517, Validation Accuracy: 88.89%
Epoch 10/20, Loss: 0.0666, Validation Accuracy: 94.44%
Epoch 11/20, Loss: 0.1550, Validation Accuracy: 100.00%
Epoch 12/20, Loss: 0.0625, Validation Accuracy: 88.89%
Epoch 13/20, Loss: 0.0678, Validation Accuracy: 94.44%
Epoch 14/20, Loss: 0.0891, Validation Accuracy: 88.89%
Epoch 15/20, Loss: 0.0626, Validation Accuracy: 88.89%
Epoch 16/20, Loss: 0.0508, Validation Accuracy: 94.44%
Epoch 17/20, Loss: 0.0717, Validation Accuracy: 88.89%
Epoch 18/20, Loss: 0.0800, Validation Accuracy: 88.89%
Epoch 19/20, Loss